In [0]:
from time import sleep
import requests
import re
import json
import datetime
import gzip


dbutils.widgets.text('account', '')
dbutils.widgets.text('startdate', '')
dbutils.widgets.text('enddate', '')
dbutils.widgets.text('marketplace', '')

account = dbutils.widgets.get('account')
startdate = dbutils.widgets.get('startdate')
enddate = dbutils.widgets.get('enddate')
marketplace = dbutils.widgets.get('marketplace')

# Customer Key and Customer Secret Key are same for all account
C_KEY = 'amzn1.application-oa2-client.c478........b0b3'
C_SECRET_KEY = 'd12827396da1f8........67c4'
C_REFRESH_TOKEN = ''

# Retrieve ProfileIds of by using https://advertising-api-eu.amazon.com/v2/profiles endpoint
# Account1 
if account == 'Account1':
    C_REFRESH_TOKEN = 'Atzr|IwEB.........R7r3bXFg'
    if marketplace == 'DE':
        ProfileId = '2169250159944659'   
    elif marketplace == 'IT':
        ProfileId = '2155969205123194' 
    elif marketplace == 'FR':
        ProfileId = '1183680406966079'    
    elif marketplace == 'NL':
        ProfileId = '3717916644632014'        
    elif marketplace == 'ES':
        ProfileId = '121977915853006'        
    elif marketplace == 'UK':
        ProfileId = '3411391515577547'  

        
# Account2 
elif account == 'Account2':
    C_REFRESH_TOKEN = 'Atzr|IwE........RC7cOJ'
    if marketplace == 'DE':
        ProfileId = '1722443589807669'
    elif marketplace == 'FR':
        ProfileId = '2182027654244981'
    elif marketplace == 'ES':
        ProfileId = '2593565687415862'
    elif marketplace == 'UK':
        ProfileId = '3143856789969928' 

In [0]:
# Get Access Token
def get_access_token():
    global refresh_token
    url = "https://api.amazon.co.uk/auth/o2/token"

    body = {
            "grant_type":"refresh_token",
            "client_id": C_KEY,
            "client_secret": C_SECRET_KEY,
            "refresh_token": C_REFRESH_TOKEN
           }
 
    response = requests.post(url = url, data = body)
    return response.json()['access_token']

C_ACCESS_TOKEN = get_access_token()

In [ ]:
# Generate Report
url = "https://advertising-api-eu.amazon.com/reporting/reports"

headers = {
            'Authorization'                   : 'Bearer ' + C_ACCESS_TOKEN,
            'Amazon-Advertising-API-ClientId' :  C_KEY,
            'Amazon-Advertising-API-Scope'    :  ProfileId,
            'Content-Type'                    : 'application/vnd.createasyncreportrequest.v3+json'
            }

body = {
          "name": "Sponsored_Products",
          "startDate": startdate,
          "endDate": enddate,
          "configuration":{
                            "adProduct": "SPONSORED_PRODUCTS",
                            "columns": [
                                        "date", "campaignId", "campaignName","adGroupId","adGroupName","adId","portfolioId","advertisedAsin", "advertisedSku",
                                        "impressions","clicks","costPerClick","clickThroughRate", "cost","spend", "campaignBudgetCurrencyCode", "campaignBudgetAmount",
                                        "campaignBudgetType", "campaignStatus", "purchases1d","purchases7d","purchases14d","purchases30d","sales1d","sales7d", "sales14d","sales30d",
                                        "attributedSalesSameSku1d","attributedSalesSameSku7d","attributedSalesSameSku14d","attributedSalesSameSku30d","unitsSoldSameSku1d",
                                        "unitsSoldSameSku7d","unitsSoldSameSku14d","unitsSoldSameSku30d","unitsSoldOtherSku7d","acosClicks7d","acosClicks14d","roasClicks7d","roasClicks14d"
                                       ],
                            "reportTypeId": "spAdvertisedProduct",
                            "format": "GZIP_JSON",
                            "groupBy": ["advertiser"],
                            "timeUnit": "DAILY"
                          }
        }

body = json.dumps(body)
generate = requests.post(url = url, headers = headers, data = body)
reportId = generate.json()['reportId']
print(reportId)

In [0]:
# Get the status and URL of the report
sleep(900)
url = f"https://advertising-api-eu.amazon.com/reporting/reports/{reportId}"

headers = {
            'Authorization'                   : 'Bearer ' + C_ACCESS_TOKEN,
            'Amazon-Advertising-API-ClientId' :  C_KEY,
            'Amazon-Advertising-API-Scope'    :  ProfileId,
            'Content-Type'                    : 'application/vnd.createasyncreportrequest.v3+json'
            }

result = requests.get(url = url, headers = headers)
report_url = result.json()['url']

In [0]:
# Downlaod the report
url = report_url
report = requests.get(url = url)
zipfile = report.content
file = (gzip.decompress(zipfile).decode('utf-8'))

In [0]:
# Connect to Blob Storage and Mounting. 
# sas token can be taken from the container settings > shared access tokens.
# Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error. 
containerName = "amazon-api-sp-all"
storageAccountName = "blobads"
sas = "sp=racwdli&st=2023-01-09T22:45:32..................xtI%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/amazon-api-sp-all")
except:
    pass
    
try:
    dbutils.fs.mount( source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
                      mount_point = "/mnt/amazon-api-sp-all",
                      extra_configs = {config : sas}
                    )
except:
    pass

In [ ]:
dbutils.fs.put('dbfs:/mnt/amazon-api-sp-all/' + 'SP__' + account + '__' + startdate + '__' + enddate + '__' + marketplace + '.json', file, overwrite = True)